In [1]:
from poloniex import Poloniex

polo = Poloniex()

In [20]:
currencies = polo.returnCurrencies()
coins = []
for name, thinks in currencies.items():
    if thinks["delisted"] == 0:
        coins.append(name)
len(coins)

69

In [28]:
# funciones necesarias para el manejo del tiempo
from datetime import datetime, timedelta
from time import sleep
from tusp import string2ts
from bot import prepareData


class PairData:
    
    def __init__(self, pair, period):
        
        self.pair = pair
        self.period = period

        
        len_data = 500
        
        tf = datetime.now()
        self.tf = tf
        end = string2ts(tf.strftime('%Y-%m-%d %H:%M:%S'))
        self.end = end
        delta = timedelta(seconds = period)
        self.delta = delta
        to = tf-delta*len_data
        self.to = to
        start = string2ts(to.strftime('%Y-%m-%d %H:%M:%S'))
        self.start = start
        
        df = prepareData(pair=pair, start=start, end=end, period=int(period))
        
        self.df = df
    
#    def sortino(self):


DGB = PairData("BTC_DGB", 3600*4)

                        close      high       low      open      quoteVolume  \
date                                                                           
2017-10-07 20:00:00  0.000003  0.000003  0.000003  0.000003   12816943.64048   
2017-10-08 00:00:00  0.000003  0.000003  0.000003  0.000003  11908337.929568   
2017-10-08 04:00:00  0.000003  0.000003  0.000003  0.000003  9643637.8590486   
2017-10-08 08:00:00  0.000003  0.000003  0.000003  0.000003  15985025.377268   
2017-10-08 12:00:00  0.000003  0.000003  0.000003  0.000003  38519566.889044   

                         volume  weightedAverage volatility  
date                                                         
2017-10-07 20:00:00   38.505898         0.000003      False  
2017-10-08 00:00:00   35.898553         0.000003      False  
2017-10-08 04:00:00   28.771210         0.000003      False  
2017-10-08 08:00:00   47.086722         0.000003       True  
2017-10-08 12:00:00  109.114193         0.000003       True  
